In [1]:
#default_exp spot

# SPOT

> Working through conversion to cogs, upload to object storage, stac item metadata parsing, etc. for SPOT 6/7 datasets

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export
import os
from glob import glob
import time
import numpy as np
from datetime import datetime
import json

import xmltodict
import pystac
from pystac import STAC_IO
from pystac.extensions.eo import Band
import geopandas as gpd

from sac_stac.utils import sedas_client, sedas_find_datasets, sedas_download, sedas_extract
from sac_stac.utils import cogmosaicbands
from sac_stac.utils import s3_upload_dir, s3_list_objects_paths, clean_up
from sac_stac.utils import pystac_setIO, create_uri

In [5]:
import pandas as pd

## **Preparation**: Download, cloud-optimise, upload

### Prep Function

Download and use basic gdal tools to mosaic any tiles into single images, convert to cog and upload to our object storage.

In [6]:
#export
def prep_spot(sedas_supplierId, inter_dir="/tmp/data/intermediate/", 
                  s3_bucket="public-eo-data", s3_dir="uksa-ssgp/spot/"):
    try:
        inter_dir = f"{inter_dir}{sedas_supplierId}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        scene_name = sedas_supplierId
        down_zip = f"{inter_dir}{scene_name}.zip"
        scene_dir = f"{down_zip[:-4]}/"
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {scene_name} within {inter_dir}")
        # find & download
        sedas_scene_res = sedas_client().search_product(sedas_supplierId)[0]
        sedas_download([sedas_scene_res], inter_dir)
        sedas_extract(down_zip, scene_dir)
        # sensor-specific band mosaicing and cogifying
        imgs_ms = glob(f"{scene_dir}*/*/*MS_001*/*.TIF")
        imgs_pan = glob(f"{scene_dir}*/*/*P_001*/*.TIF")
        cogmosaicbands(imgs_pan, 1, imgs_pan[0][:-20])
        cogmosaicbands(imgs_ms, 4, imgs_ms[0][:-20])
        # upload
        s3_upload_dir(scene_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {scene_name} at {s3_dir}{scene_name}/")
        clean_up(inter_dir)
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
        clean_up(inter_dir)

### Iteration with samples

Some samples used to iterate creation of the prep function.

In [7]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "SPOT"
                            )
pd.DataFrame(result['products'])

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,ff3aa1470eb5ed57d7c5a6e05db400b7,UKSA_SPOT161_SO18034609-61-01_DS_SPOT6_2018101...,ARCHIVE,SPOT-6,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-2.820905 50.931699,-1.870556 50.9345...",2018-10-19T10:39:14Z,...,4.313839e+09,0.0,0.0,6.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '63...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,c0982f29580eb726f04c9af6bdebb376,UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_2018101...,ARCHIVE,SPOT-6,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-1.918729 51.158248,-0.864598 51.1527...",2018-10-10T10:58:21Z,...,3.463299e+09,25.0,13.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '73...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
2,e60f722dcbb5d4a122a231183a66ab0e,UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_2018101...,ARCHIVE,SPOT-6,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-1.699603 51.685303,-0.646975 51.6778...",2018-10-10T10:58:10Z,...,3.325405e+09,29.0,16.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'a9...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
3,65f52dbd5981d6a203b3070927993898,UKSA_SPOT291_SO18034610-90-01_DS_SPOT7_2018092...,ARCHIVE,SPOT-7,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-2.055493 50.446539,-1.051938 50.4426...",2018-09-27T10:59:25Z,...,1.227865e+10,100.0,15.0,1.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '54...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
4,7e3fa163ffa951ec0850714139a8a073,UKSA_SPOT290_SO18034610-89-01_DS_SPOT7_2018092...,ARCHIVE,SPOT-7,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-2.796381 51.428559,-1.884609 51.4312...",2018-09-27T10:59:01Z,...,7.219406e+09,3.0,1.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4d...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
5,0898d97bb20b981a6280f4310b3556f2,UKSA_SPOT289_SO18034610-88-01_DS_SPOT7_2018092...,ARCHIVE,SPOT-7,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-2.791968 51.422924,-1.891268 51.4255...",2018-09-27T10:58:35Z,...,7.202192e+09,3.0,1.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'c8...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
6,f4b5e3c63318bbc021fff734873a4001,UKSA_SPOT288_SO18034610-87-01_DS_SPOT7_2018092...,ARCHIVE,SPOT-7,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-1.288551 51.408716,-0.281425 51.3982...",2018-09-27T10:58:11Z,...,8.301192e+09,18.0,4.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'ab...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
7,b66bad8bfa0fcee1918558bbbad733e2,UKSA_SPOT287_SO18034610-86-01_DS_SPOT7_2018092...,ARCHIVE,SPOT-7,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-2.382915 51.450709,-1.22423 51.44877...",2018-09-24T10:31:59Z,...,4.208909e+09,91.0,39.0,4.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '6d...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
8,4d553fcdddf8b2059caee5c1376fca03,UKSA_SPOT134_SO18034609-34-01_DS_SPOT6_2018090...,ARCHIVE,SPOT-6,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-1.772879 51.279608,-0.816458 51

In [ ]:
for p in pd.DataFrame(result['products']).supplierId.values:
    
#     if not True in [i.split('_')[1] in p for i in done]:
    prep_spot(p)

Example of testing the first one.

In [ ]:
prep_spot("UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140")

2020-11-10 11:42:18 Preparing UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140 within /tmp/data/intermediate/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140_tmp/
2020-11-10 11:42:19 Downloading
2020-11-10 11:45:14 Downloaded
2020-11-10 11:45:14 Extracting /tmp/data/intermediate/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140_tmp/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140.zip
2020-11-10 11:47:29 Extracted to /tmp/data/intermediate/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140_tmp/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140/
2020-11-10 11:47:29 Mosaicing band 1 imgs: /tmp/data/intermediate/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140_tmp/UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140

## **STAC metadata**: core & limited extension

With some cogs hosted on object storage we can fill out some of the necessary STAC core and extension metadata fields.

### *Collection* fields

Since for SPOT 6 & 7 we're working with electro-optical imagery, we can make use of the STAC **[EO Extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/eo)**. At the SPOT *Collection* level we can therefore manually set some default metadata fields that will be consistent across all *Items* and *Assets*.

In [ ]:
#export
spot_bands = [Band.create(name='Panchromatic', description='Panchromatic: 450 - 745 nm', common_name='pan'),
              Band.create(name='Blue', description='Blue: 450 - 520 nm', common_name='blue'),
              Band.create(name='Green', description='Green: 530 - 590 nm', common_name='green'),
              Band.create(name='Red', description='Red: 625 - 695 nm', common_name='red'),
              Band.create(name='Near-Infrared', description='Near-Infrared: 760 - 890 nm', common_name='nir')]

We could also make use of the **[Satellite Extension](https://github.com/radiantearth/stac-spec/blob/master/extensions/sat/README.md)**, however the fields of orbit_state (i.e. ascending / descending) and relative_orbit (i.e. orbit no.) don't seem immediately useful for Pleiades. Other extensions are commented on as we delve deeper into a *Collection*.

### *Item* fields

In [8]:
obj_paths_list = s3_list_objects_paths('public-eo-data', 'uksa-ssgp/spot/')

In [9]:
# get unique Item / scene names (third dir from path)
scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
scene_names[:5]

['UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140',
 'UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140',
 'UKSA_SPOT161_SO18034609-61-01_DS_SPOT6_201810191039150_FR1_FR1_SV1_SV1_W002N51_01709',
 'UKSA_SPOT287_SO18034610-86-01_DS_SPOT7_201809241031594_FR1_FR1_FR1_FR1_W002N52_01222',
 'UKSA_SPOT288_SO18034610-87-01_DS_SPOT7_201809271058114_FR1_FR1_SV1_SV1_W001N52_02845']

In [10]:
# can just work with one
scene_name = scene_names[0]
scene_name

'UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140'

In [11]:
# and objects associated with that scene
scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
scene_obj_paths[:2]

['uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/DELIVERY.PDF',
 'uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/INDEX.HTM']

In [12]:
# set our own I/O for pystac
pystac_setIO()

We have a set of pleiades-specific tools for parsing stac metadata at the *Item* level. Some are more complex than others both in approach for generating and assumptions.

We start off simple by getting the date from the scene name.

In [13]:
#export
def spot_get_dt(scene_name):
    return datetime.strptime(scene_name.split('_')[5][:14], '%Y%m%d%H%M%S')

In [14]:
spot_get_dt(scene_name)

datetime.datetime(2018, 10, 10, 10, 58, 9)

In [15]:
scene_name

'UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140'

A variety of *Item* metadata already exists within either the multi-spectral and / or the panchromatic image metadata supplied with the original dataset.

In [16]:
#export
def spot_parsemeta(meta_path):
    return xmltodict.parse(pystac.STAC_IO.read_text(create_uri(meta_path)))

Typically there are two metadata files, one each for both MS & PAN, but both offer equivalent info for our purposes.

In [17]:
meta = spot_parsemeta([i for i in scene_obj_paths if (i.endswith('.XML')) & (os.path.basename(i).startswith('DIM'))][0]) # not sure if this catches for all Items?

Get crs. Typically only used for eo extension, but helps transform of the geom into 4326.

In [18]:
def spot_get_crs(metadata):
    return int(metadata['Dimap_Document']['Coordinate_Reference_System']['Projected_CRS']['PROJECTED_CRS_NAME'][:5])

In [19]:
spot_get_crs(meta)

27700

Geometry (footprint of actually useful data) can be taken from a .gml, but need to set crs on load sometimes for the reprojection, so parse from earlier.

In [51]:
#export
def spot_get_geom(scene_paths, native_epsg):
    nat_crs = {"init": f"epsg:{native_epsg}"}
    roi_path = [i for i in scene_paths if (i.endswith('1_MSK.GML') * os.path.basename(i).startswith('ROI'))][0]
    roi_uri = create_uri(roi_path)
    g = gpd.read_file(roi_uri)
    g.crs = f"EPSG:{native_epsg}"
    return json.loads(g.to_crs('EPSG:4326').to_json(show_bbox=True))['features'][0]['geometry']

In [52]:
spot_get_geom(scene_obj_paths, '27700')

{'type': 'Polygon',
 'coordinates': [[[-0.6499519273083667, 52.08844089007527],
   [-0.6448428681559101, 52.088382273153236],
   [-0.6446263981696051, 51.757267668413746],
   [-0.644682404135545, 51.755396785700185],
   [-0.6450817810991609, 51.75512292935542],
   [-1.6964988848377145, 51.685308252199924],
   [-1.6970101211854633, 51.685309570681724],
   [-1.6975136320308248, 51.685588611054996],
   [-1.6977238663030947, 51.85203916138853],
   [-1.6978394095956657, 51.94347348226774],
   [-1.6973325231656649, 52.018813144847385],
   [-0.6499519273083667, 52.08844089007527]]]}

Bounding box is one..

In [54]:
#export
def spot_get_bbox(metadata):
    lons = [float(i['LON']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    lats = [float(i['LAT']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    return [min(lons), min(lats), max(lons), max(lats)]

In [55]:
spot_get_bbox(meta)

[-1.69960277718, 51.6779380171, -0.634664143202, 52.095806321]

In [56]:
item = pystac.Item(id=scene_name,
                  datetime=spot_get_dt(scene_name),
                  geometry=spot_get_geom(scene_obj_paths, native_epsg),
                  bbox=spot_get_bbox(meta),
                  properties={})

In [57]:
item.validate()

In [65]:
def spot_get_gsd(metadata):
    across = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ACROSS_TRACK'])
    along = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ALONG_TRACK'])
    return round(( across + along ) / 2, 2)

In [67]:
spot_get_gsd(meta)

9.93

In [69]:
item.common_metadata.gsd = spot_get_gsd(meta)

In [70]:
item.ext.enable('eo')

In [71]:
def spot_get_cloudcover(metadata):
    return round(float(metadata['Dimap_Document']['Dataset_Content']['CLOUD_COVERAGE']['#text']),2)

In [72]:
spot_get_cloudcover(meta)

0.0

In [74]:
item.ext.eo.cloud_cover = spot_get_cloudcover(meta)

In [75]:
item.ext.enable('projection')

In [76]:
item.ext.projection.epsg = spot_get_crs(meta)


In [77]:
item.validate()

### Adding *Assets*

In [80]:
#export
spot_bands = [Band.create(name='Panchromatic', description='Panchromatic: 450 - 745 nm', common_name='pan'),
              Band.create(name='Blue', description='Blue: 450 - 520 nm', common_name='blue'),
              Band.create(name='Green', description='Green: 530 - 590 nm', common_name='green'),
              Band.create(name='Red', description='Red: 625 - 695 nm', common_name='red'),
              Band.create(name='Near-Infrared', description='Near-Infrared: 760 - 890 nm', common_name='nir')]

In [78]:
bfs = {
    'Panchromatic':{'ends':'_band1', 'dif':'_P_', 'id':'B0'},
    'Blue':{'ends':'_band1', 'dif':'_MS_', 'id':'B1'},
    'Green':{'ends':'_band2', 'dif':'_MS_', 'id':'B2'},
    'Red':{'ends':'_band3', 'dif':'_MS_', 'id':'B3'},
    'Near-Infrared':{'ends':'_band4', 'dif':'_MS_', 'id':'B4'}
}
bfs

{'Panchromatic': {'ends': '_band1', 'dif': '_P_', 'id': 'B0'},
 'Blue': {'ends': '_band1', 'dif': '_MS_', 'id': 'B1'},
 'Green': {'ends': '_band2', 'dif': '_MS_', 'id': 'B2'},
 'Red': {'ends': '_band3', 'dif': '_MS_', 'id': 'B3'},
 'Near-Infrared': {'ends': '_band4', 'dif': '_MS_', 'id': 'B4'}}

In [82]:
for band in spot_bands:
    print(band.name)
    
    matched_paths = [ o for o in scene_obj_paths if (o.endswith(f"{bfs[band.name]['ends']}.tif")) & (f"{bfs[band.name]['dif']}" in o) ]
    if len(matched_paths) > 1:
        raise Exception(f"Found too many matches: {matched_paths}")
    band_url = create_uri(matched_paths[0])
    print(band_url)
    
    asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
    item.ext.eo.set_bands([band], asset)
    item.add_asset(bfs[band.name]['id'], asset)    

Panchromatic
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_P_001_A/IMG_SPOT6_P_201810101058095_ORT_band1.tif
Blue
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_MS_001_A/IMG_SPOT6_MS_201810101058095_ORT_band1.tif
Green
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_MS_001_A/IMG_SPOT6_MS_201810101058095_ORT_band2.tif
Red
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_MS_001_A/IMG_SPOT6_MS_201810101058095_ORT_band3.tif
Near-Infra

In [83]:
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140',
 'properties': {'datetime': '2018-10-10T10:58:09Z',
  'gsd': 9.93,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-0.6499519273083667, 52.08844089007527],
    [-0.6448428681559101, 52.088382273153236],
    [-0.6446263981696051, 51.757267668413746],
    [-0.644682404135545, 51.755396785700185],
    [-0.6450817810991609, 51.75512292935542],
    [-1.6964988848377145, 51.685308252199924],
    [-1.6970101211854633, 51.685309570681724],
    [-1.6975136320308248, 51.685588611054996],
    [-1.6977238663030947, 51.85203916138853],
    [-1.6978394095956657, 51.94347348226774],
    [-1.6973325231656649, 52.018813144847385],
    [-0.6499519273083667, 52.08844089007527]]]},
 'links': [],
 'assets': {'B0': {'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/spot/UKSA_SPOT155_SO18034609